# Visualisation d'une série temporelle d'images sur Toulouse

Nous allons appliquer ce que nous avons vu précédemment pour charger et visualise une série temporelle d'images.

Cela va aussi être l'occasion de faire la démonstration d'outils de visualisation avancés sur jupyter

In [ ]:
!pip install leafmap requests localtileserver geogif

In [ ]:
import os
from pathlib import Path

import requests

image_urls = [
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201504131053028_8bits.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201506041053076_8bits.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201609071056565_8bits.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201707041049221_8bits.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201908041045469_8bits.tif",
]


def download_file(in_url: str, out_filename: Path):
    if not (out_filename).exists():
        out_filename.parent.mkdir(exist_ok=True)
        print("Downloading", in_url)
        response = requests.get(in_url)
        with open(out_filename, "wb") as f:
            f.write(response.content)


for image_url in image_urls:
    download_file(image_url, Path() / "data" / image_url.split("/")[-1])

In [ ]:
image_files = sorted(list(Path("data").glob("*.tif")))
image_files

## Chargement d'une série temporelle et quelques traitements

In [ ]:
import datetime

import geogif
import numpy as np
import xarray as xr

In [ ]:
def path_to_datetime(image_file):
    date = image_file.stem
    date = date.split("_")[-2][:8]
    date = datetime.datetime.strptime(date, "%Y%m%d")
    return date


timestamps = []
timeseries = []

for image_file in image_files:
    timestamps.append(path_to_datetime(image_file))
    img = xr.open_rasterio(image_file, chunks=dict(x=512, y=512))
    img = img.assign_coords(coords=dict(band=["Red", "Green", "Blue", "Nir"]))
    timeseries.append(img)

time_var = xr.Variable("time", timestamps)

timeseries = xr.concat(timeseries, dim=time_var)

In [ ]:
timeseries

In [ ]:
# Notons que la série temporelle est "lazy" car on a utilisé l'option "chunks" de open_rasterio
type(timeseries.data)

Affichage de la série temporelle via xarray

In [ ]:
timeseries.sel(band=["Red", "Green", "Blue"]).plot.imshow(col="time", robust=True)

In [ ]:
timeseries.sel(band=["Nir", "Green", "Blue"]).plot.imshow(col="time", robust=True)

Zoom sur une vignette particulière de l'image globale

In [ ]:
area_of_interest = timeseries[:, :, 3100:3400, 2200:2500]
area_of_interest

In [ ]:
area_of_interest.sel(band=["Red", "Green", "Blue"]).plot.imshow(col="time", robust=True)

In [ ]:
# On peut maintenant faire un gif sans tomber "out of memory"
my_gif = geogif.dgif(area_of_interest.sel(band=["Red", "Green", "Blue"]), fps=1)

In [ ]:
my_gif.compute()

Calcul du NDVI

In [ ]:
red = area_of_interest.sel(band=["Red"]).astype(np.float32) / 255.0
nir = area_of_interest.sel(band=["Nir"]).astype(np.float32) / 255.0
red = red.assign_coords(coords=dict(band=["ndvi"]))
nir = nir.assign_coords(coords=dict(band=["ndvi"]))

In [ ]:
# Notons encore une fois que le calcul n'est pas encore effectué
red

In [ ]:
nir

In [ ]:
ndvi = (nir - red) / (nir + red)

In [ ]:
ndvi

In [ ]:
ndvi = ndvi.sel(band=["ndvi"]).compute()

In [ ]:
geogif.gif(ndvi, fps=1, cmap="viridis")

## Visualisation interactive d'une image

Démonstration de ipyleaflet et de localtileserver 

Si ça ne fonctionne pas, lancez: 

```bash
jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
```

dans un terminal du jupyterlab


In [ ]:
from ipyleaflet import LayersControl, Map, Marker, ScaleControl, FullScreenControl, SplitMapControl
from localtileserver import get_leaflet_tile_layer

In [ ]:
image = get_leaflet_tile_layer(image_files[0], band=[1, 2, 3], name=image_files[0].stem.replace("_8bits", ""))
supaero = Marker(location=(43.56588645818575, 1.4746516942977905), name="ISAE")

# Create ipyleaflet map, add tile layer, and display
m = Map(center=(43.585, 1.455), zoom=13)
m.add_control(ScaleControl(position="bottomleft"))
m.add_control(LayersControl(position="topright"))
m.add_layer(image)
m.add_layer(supaero)

In [ ]:
m

## Visualisation comparative de deux images

Observons les changements sur la zone de Toulouse entre 2015 et 2016

In [ ]:
# Create ipyleaflet tile layer from that server
image_1 = get_leaflet_tile_layer(image_files[0], band=[1, 2, 3], name=image_files[0].stem.replace("_8bits", ""))
image_2 = get_leaflet_tile_layer(image_files[-1], band=[1, 2, 3], name=image_files[-1].stem.replace("_8bits", ""))
supaero = Marker(location=(43.56588645818575, 1.4746516942977905), name="ISAE")

# Create ipyleaflet map, add tile layer, and display
m = Map(center=(43.585, 1.455), zoom=13)
m.add_control(ScaleControl(position="bottomleft"))
m.add_control(FullScreenControl())
m.add_layer(supaero)
control = SplitMapControl(left_layer=image_1, right_layer=image_2)
m.add_control(control)

In [ ]:
m

Essayez de comparer la même image en R-G-B (bandes 1,2,3) et NIR-G-G (bandes 4,2,3)

In [ ]:
# TODO